In [75]:
# This will connect to a postgres database using the public IP address of the VM
# The bronze tables to create the silver tables

import os 
import sqlalchemy
import pandas as pd
from shapely.geometry import Point, Polygon
import datetime
import psycopg2
import pandas as pd
import geopandas as gpd


In [76]:

user = 'postgres'
password = '432'
ip = '34.134.248.227'
db_name = 'bronze'
port = '5432'



In [77]:
def get_zip(lat, long):
    '''
    Gets the zip code comparing the latitude and longitude to the zips.geojson file
    
    '''
    point = Point(long, lat)
    for i in range(len(zips)):
        if point.within(zips['geometry'][i]):
            return zips['zip'][i]
        else:
            continue


In [78]:
# connect to database
conn = psycopg2.connect(f'host={ip} dbname={db_name} user={user} password={password} port={port}')

# select latitude and longitude from the table and run
sql = "SELECT trip_start_timestamp as ride_date, dropoff_centroid_latitude as lat, dropoff_centroid_longitude as long, trip_id FROM taxi_trips;"
df_taxi = pd.read_sql(sql, conn)
sql = "SELECT trip_start_timestamp as ride_date, dropoff_centroid_latitude as lat, dropoff_centroid_longitude as long, trip_id  FROM tnp_trips;"
df_tnp = pd.read_sql(sql,conn)

# load zips.geojson file with geopandas
zips = gpd.read_file('zips.geojson')



In [79]:
# Takes the latitude and longitude from df and create a zip code column
# Uses reverse_geocoder to find the zip code for each latitude and longitude using the zips.geojson file

zip_list = [get_zip(lat_long[1]['lat'],lat_long[1]['long']) for lat_long in df_taxi[['long','lat']].iterrows()]
df_taxi['zip'] = zip_list
df_taxi['ride_type'] = 'taxi'

zip_list = [get_zip(lat_long[1]['lat'],lat_long[1]['long']) for lat_long in df_tnp[['long','lat']].iterrows()]
df_tnp['zip'] = zip_list
df_tnp['ride_type'] = 'rideshare'

# append the datasets
df_ride = df_taxi.append(df_tnp, ignore_index=True, sort='ride_date')

# get the week number, so we can join on the weekly covid info
df_ride['week_number'] = [int(item.strftime("%W")) for item in df_ride['ride_date']]
df_ride['year'] = [item.strftime('%Y') for item in df_ride['ride_date']]

# df.to_csv('./src/python/df.csv', encoding='utf-8', index=False)

In [80]:
# now for the covid data
sql_query = '''SELECT zip_code as zip, week_number, cases_weekly, week_start,
                case_rate_cumulative, deaths_weekly, death_rate_weekly 
                from covid_19_zip;'''
df_covid_zip = pd.read_sql(sql_query, conn)
df_covid_zip.rename(columns={'cases_weekly':'covid_rate'}, inplace=True)
df_covid_zip.rename(columns={'death_rate_weekly':'death_rate'}, inplace=True)


df_covid_zip['year'] = [item.strftime('%Y') for item in df_covid_zip['week_start']]

In [83]:
# and to join on the zip and week_number
df_taxi_covid = pd.merge(df_ride, df_covid_zip, how= 'left', on=['year','week_number','zip'])